In [6]:
import re

def read_input(filename):
    with open(filename, 'r') as file:
        alpha = file.readline().strip()
        num_clauses = int(file.readline().strip())
        # Handle 'or' and 'and' operators
        clauses = [re.split(r'\s+OR\s+', clause) for clause in file.readlines()]
        clauses = [set(literal.strip() for literal in clause) for clause in clauses]
    print(f"Perform read_input {filename}")
    print('alpha: ', alpha)
    print('clauses: ', clauses)
    print("--------------------")
    return alpha, clauses

def write_output(filename, results):
    print(f"Perform write_output {filename} with the results: {results}")
    print("--------------------")
    with open(filename, 'w') as file:
        for loop_result in results:
            file.write(f"{len(loop_result)}\n")
            for clause in loop_result:
                # Join literals in the clause with ' OR '
                formatted_clause = ' OR '.join(clause)
                file.write(f"{formatted_clause}\n")
            file.write("YES" if len(loop_result) == 0 else "NO")
            file.write("\n")  # Add a new line after "YES" or "NO"

def pl_resolution(alpha, kb_clauses, output_filename):
    results = []
    neg_alpha = f"-{alpha}"

    # Initial loop
    current_loop = set(frozenset(clause) for clause in kb_clauses)  # Convert to a set of frozensets
    unique_clauses = set(current_loop)  # Set to keep track of unique clauses
    results.append(current_loop)

    while True:
        new_clauses = set()

        for ci, clause_i in enumerate(current_loop):
            for cj, clause_j in enumerate(current_loop):
                if ci < cj:
                    resolvents = resolve(clause_i, clause_j)
                    new_clauses.update(resolvents)

        if frozenset() in new_clauses:
            break  # Empty clause found, KB entails alpha

        if new_clauses.issubset(current_loop):
            break  # No new clauses can be generated

        # Only update if new clauses are encountered
        unique_new_clauses = new_clauses - unique_clauses
        if unique_new_clauses:
            unique_clauses.update(unique_new_clauses)
            current_loop.update(unique_new_clauses)
            results.append(current_loop.copy())

            # Write the output file at the end of each loop
            write_output(output_filename, results)

    print(f"Perform pl_resolution with alpha: {alpha} and kb_clauses: {kb_clauses}")
    print('results: ', results)
    print("--------------------")
    return results

def resolve(clause_i, clause_j):
    resolvents = set()

    for literal in clause_i:
        neg_literal = f"-{literal}"
        if neg_literal in clause_j:
            resolvent = (clause_i - {literal}) | (clause_j - {neg_literal})
            resolvent -= {""}  # Remove empty literals using the '-' operator
            resolvents.add(frozenset(resolvent))  # Convert to frozenset

    print(f"Perform resolve with clause_i: {clause_i} and clause_j: {clause_j}")
    print('resolvents: ', resolvents)
    print("--------------------")
    return resolvents

def is_similar_clause(clause):
    # Check if the clause has "A or B or -B" format and "True" values
    positive_literals = {literal for literal in clause if not literal.startswith('-')}
    negative_literals = {literal[1:] for literal in clause if literal.startswith('-')}
    
    # Check if the set of positive literals is a subset of the set of negative literals
    return positive_literals.issubset(negative_literals)

def main(input_filename, output_filename):
    alpha, kb_clauses = read_input(input_filename)
    neg_alpha = f"-{alpha}"

    pl_resolution(neg_alpha, kb_clauses, output_filename)

# Example usage:
main("input.txt", "output.txt")


Perform read_input input.txt
alpha:  -A
clauses:  [{'B', '-A'}, {'B', '-C'}, {'C', 'A', '-B'}, {'-B'}]
--------------------
Perform resolve with clause_i: frozenset({'B', '-C'}) and clause_j: frozenset({'C', 'A', '-B'})
resolvents:  {frozenset({'-C', 'A', 'C'})}
--------------------
Perform resolve with clause_i: frozenset({'B', '-C'}) and clause_j: frozenset({'B', '-A'})
resolvents:  set()
--------------------
Perform resolve with clause_i: frozenset({'B', '-C'}) and clause_j: frozenset({'-B'})
resolvents:  {frozenset({'-C'})}
--------------------
Perform resolve with clause_i: frozenset({'C', 'A', '-B'}) and clause_j: frozenset({'B', '-A'})
resolvents:  {frozenset({'B', 'C', '-B'})}
--------------------
Perform resolve with clause_i: frozenset({'C', 'A', '-B'}) and clause_j: frozenset({'-B'})
resolvents:  set()
--------------------
Perform resolve with clause_i: frozenset({'B', '-A'}) and clause_j: frozenset({'-B'})
resolvents:  {frozenset({'-A'})}
--------------------
Perform write_